# DFT + PCM ring-strain evaluation (from CORE_SET generator outputs)

This notebook is a **drop-in replacement** for `ring_strain_dft_pcm_all_sizes.ipynb`,
but it reads the **structures produced by** `ring_strain_core_set_generate_plot.ipynb`
(i.e., the `manifest.json` and the multi-frame `*_topK.xyz` files).

For each manifest entry (molecule + cut bond), it computes **DFT(+PCM) energies**
for the four species:

- `M`      : macrocycle
- `M_open` : cut + capped macrocycle
- `X`      : local reference fragment
- `X_open` : cut + capped local fragment

and then computes the local-reference ring strain proxy:
\[
E_{\mathrm{strain}} = (E_{M_{\mathrm{open}}}-E_M) - (E_{X_{\mathrm{open}}}-E_X).
\]

### Notes
- The CORE_SET generator writes **multi-frame XYZ** files (`*_topK.xyz`).  
  By default we evaluate only the **first frame** for speed, but you can switch to
  `conformer_policy="min"` to compute several frames and take the lowest DFT energy.
- Results are saved incrementally so you can interrupt/resume.


In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
from typing import Optional

import rdkit
print("RDKit", rdkit.__version__)

from pyscf import gto, dft
print("PySCF imported")


RDKit 2025.09.3
PySCF imported


In [3]:
# --- Minimal sanity-check cell: DFT vs DFT+solvent (PCM) on H2O ---
# If this runs and prints two different energies, your DFT+solvent plumbing is working.

try:
    from pyscf import gto, dft
    from pyscf.solvent import pcm, ddcosmo
except Exception as e:
    raise ImportError(
        "PySCF not found in this environment. Install it (e.g., `pip install pyscf`) "
        "then rerun this cell."
    ) from e


def attach_solvent(mf, solvent_name="water", eps=78.3553):
    """
    Try a few PySCF solvent backends (API varies slightly across versions).
    Returns an SCF/DFT object with a continuum solvent model attached.
    """
    # Newer PySCF builds sometimes expose a convenience method
    if hasattr(mf, "PCM"):
        try:
            return mf.PCM(solvent=solvent_name)
        except TypeError:
            # some versions accept no kwargs, or use different names
            try:
                return mf.PCM()
            except Exception:
                pass

    # PCM module
    try:
        return pcm.PCM(mf, solvent=solvent_name)
    except Exception:
        pass

    # ddCOSMO fallback (not exactly PCM, but still continuum solvation)
    try:
        return ddcosmo.DDCOSMO(mf, eps=eps)
    except Exception as e:
        raise RuntimeError("Could not attach any solvent model (PCM or ddCOSMO).") from e


# 1) Build a tiny molecule (water) in Angstrom
mol = gto.M(
    atom="""
    O  0.000000  0.000000  0.000000
    H  0.000000 -0.757160  0.586260
    H  0.000000  0.757160  0.586260
    """,
    basis="def2-svp",
    charge=0,
    spin=0,
    unit="Angstrom",
    verbose=4,
)

# 2) Gas-phase DFT
mf_gas = dft.RKS(mol)
mf_gas.xc = "wb97x"
mf_gas.grids.level = 3
mf_gas.conv_tol = 1e-10
e_gas = mf_gas.kernel()

# 3) Solvated DFT (continuum solvent)
mf_sol = dft.RKS(mol)
mf_sol.xc = "wb97x"
mf_sol.grids.level = 3
mf_sol.conv_tol = 1e-10
mf_sol = attach_solvent(mf_sol, solvent_name="water")  # try PCM, else ddCOSMO
e_sol = mf_sol.kernel()

print("\n=== Energies (Hartree) ===")
print(f"Gas-phase      : {e_gas:.12f}")
print(f"With solvent    : {e_sol:.12f}")
print(f"Solvation shift : {e_sol - e_gas:+.12f} Ha   ({(e_sol - e_gas)*627.509474:+.3f} kcal/mol)")


System: uname_result(system='Linux', node='thomas-watts-LEGION-T5-26IRB8', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 17:35:02) [GCC 11.2.0]
numpy 1.26.4  scipy 1.16.3  h5py 3.15.1
Date: Fri Dec 19 15:29:26 2025
PySCF version 2.11.0
PySCF path  /home/thomas-watts/miniconda3/envs/dvr_H/lib/python3.12/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = Angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.0000000000

In [ ]:
# ----------------------------
# Inputs: where CORE_SET wrote outputs
# ----------------------------
core_out_root = Path("ring_strain_outputs_core_set")  # <-- change if needed
manifest_path = core_out_root / "manifest.json"

# Optional filters (set to None to run everything in manifest)
NAME_FILTER = None     # e.g. "CP1"
BOND_FILTER = None     # e.g. {12, 37} or list like [12, 37]

# ----------------------------
# DFT(+PCM) settings
# ----------------------------
xc = "b3lyp"
basis = "def2-svp"
charge = 0
spin = 0            # 2S; singlet = 0

# Solvent / PCM
pcm_method = "IEF-PCM"
eps = 36.7          # DMF; set 78.3553 for water

# SCF controls
max_cycle = 200
conv_tol = 1e-9

# ----------------------------
# Multi-frame XYZ handling
# ----------------------------
conformer_policy = "first"   # "first" (fast) or "min" (evaluate multiple frames and take minimum)
max_frames = 5               # only used if conformer_policy == "min"


In [ ]:
# ----------------------------
# Load manifest produced by the CORE_SET generator
# ----------------------------
manifest = json.loads(manifest_path.read_text())
print("Loaded entries:", len(manifest))

def _entry_name(e):
    # CORE_SET manifest uses 'name'; other manifests may use 'ring'
    return e.get("name", e.get("ring", "UNKNOWN"))

def _entry_bond(e):
    return int(e.get("bond_idx", e.get("bond", e.get("bond_index", -1))))

bond_set = None
if BOND_FILTER is not None:
    bond_set = BOND_FILTER if isinstance(BOND_FILTER, set) else set(BOND_FILTER)

to_run = []
for e in manifest:
    nm = _entry_name(e)
    b  = _entry_bond(e)
    if (NAME_FILTER is not None) and (nm != NAME_FILTER):
        continue
    if (bond_set is not None) and (b not in bond_set):
        continue
    to_run.append(e)

print("Entries to run:", len(to_run))
print("Example entry keys:", sorted(list(to_run[0].keys())) if to_run else "None")


In [ ]:
# ----------------------------
# XYZ utilities (supports multi-frame XYZ)
# ----------------------------
def read_xyz_frames(xyz_path: Path, max_frames: 'Optional[int]' = None):
    """Return a list of (comment, atoms) frames from an XYZ file.

    atoms: list[(sym, x, y, z)]
    """
    lines = xyz_path.read_text().splitlines()
    i = 0
    frames = []
    while i < len(lines):
        # skip empty lines
        while i < len(lines) and (not lines[i].strip()):
            i += 1
        if i >= len(lines):
            break
        n = int(lines[i].strip())
        if i + 1 >= len(lines):
            break
        comment = lines[i+1].strip()
        atoms = []
        for ln in lines[i+2:i+2+n]:
            parts = ln.split()
            if len(parts) < 4:
                continue
            s, x, y, z = parts[:4]
            atoms.append((s, float(x), float(y), float(z)))
        frames.append((comment, atoms))
        i = i + 2 + n
        if (max_frames is not None) and (len(frames) >= max_frames):
            break
    if not frames:
        raise ValueError(f"No frames parsed from {xyz_path}")
    return frames

def atoms_to_pyscf_atom(atoms):
    return "\n".join([f"{s} {x} {y} {z}" for (s, x, y, z) in atoms])


In [ ]:
# ----------------------------
# PCM attachment + DFT energy
# ----------------------------
def attach_pcm(mf, method: str, eps: float):
    """Attach PCM using mf.PCM() when available; otherwise fall back to pyscf.solvent.pcm.PCM."""
    if hasattr(mf, "PCM"):
        mf2 = mf.PCM()
    else:
        from pyscf.solvent import pcm
        mf2 = pcm.PCM(mf)
    mf2.with_solvent.method = method
    mf2.with_solvent.eps = float(eps)
    return mf2

def dft_pcm_energy_from_atoms(atoms,
                             xc: str, basis: str, charge: int, spin: int,
                             pcm_method: str, eps: float,
                             max_cycle: int = 200, conv_tol: float = 1e-9):
    mol = gto.M(
        atom=atoms_to_pyscf_atom(atoms),
        basis=basis,
        charge=int(charge),
        spin=int(spin),
        unit="Angstrom",
        verbose=0,
    )
    mf = dft.RKS(mol)
    mf.xc = xc
    mf.max_cycle = int(max_cycle)
    mf.conv_tol = float(conv_tol)
    mf = attach_pcm(mf, method=pcm_method, eps=eps)
    e = mf.kernel()
    return float(e)

def dft_pcm_energy_from_xyz(xyz_path: Path,
                            conformer_policy: str = "first",
                            max_frames: int = 5,
                            **kwargs):
    """Compute DFT(+PCM) energy from an XYZ path.

    Returns:
      (E_best, best_frame_idx, n_frames_considered)
    """
    if conformer_policy not in {"first", "min"}:
        raise ValueError("conformer_policy must be 'first' or 'min'")
    frames = read_xyz_frames(xyz_path, max_frames=(1 if conformer_policy=="first" else max_frames))
    energies = []
    for fi, (comment, atoms) in enumerate(frames):
        e = dft_pcm_energy_from_atoms(atoms, **kwargs)
        energies.append(e)
    energies = np.array(energies, dtype=float)
    best_idx = int(np.argmin(energies))
    return float(energies[best_idx]), best_idx, int(len(energies))


In [ ]:
# ----------------------------
# Run + incremental save (resume-friendly)
# ----------------------------
species_list = ["M", "M_open", "X", "X_open"]

results_dir = core_out_root / "dft_pcm_core_set"
results_dir.mkdir(exist_ok=True)

per_species_csv = results_dir / "dft_pcm_per_species.csv"
summary_csv     = results_dir / "dft_pcm_summary.csv"

# Load existing partial results if present
if per_species_csv.exists():
    df_done = pd.read_csv(per_species_csv)
    print("Loaded existing per-species results:", len(df_done))
else:
    df_done = pd.DataFrame(columns=[
        "name","bond_idx","species","xyz","conformer_policy","max_frames",
        "E_Ha","best_frame","n_frames"
    ])

done_keys = set(zip(df_done["name"], df_done["bond_idx"], df_done["species"]))

rows = []
rows.extend(df_done.to_dict("records"))

def resolve_xyz_path(x):
    p = Path(x)
    if p.exists():
        return p
    # Many manifests store relative paths, relative to core_out_root
    p2 = core_out_root / p
    if p2.exists():
        return p2
    raise FileNotFoundError(f"Could not resolve xyz path: {x}")

for idx, e in enumerate(to_run):
    name = _entry_name(e)
    bond_idx = _entry_bond(e)
    sp_dict = e.get("species", {})
    print(f"[{idx+1}/{len(to_run)}] {name} bond {bond_idx}")
    for sp in species_list:
        key = (name, bond_idx, sp)
        if key in done_keys:
            continue
        xyz_rel = sp_dict.get(sp, {}).get("xyz", None)
        if xyz_rel is None:
            print(f"  - {sp}: missing xyz in manifest; skipping")
            continue
        xyz_path = resolve_xyz_path(xyz_rel)

        try:
            E, best_frame, n_frames = dft_pcm_energy_from_xyz(
                xyz_path,
                conformer_policy=conformer_policy,
                max_frames=max_frames,
                xc=xc, basis=basis, charge=charge, spin=spin,
                pcm_method=pcm_method, eps=eps,
                max_cycle=max_cycle, conv_tol=conv_tol,
            )
        except Exception as ex:
            print(f"  - {sp}: FAILED ({type(ex).__name__}: {ex})")
            E, best_frame, n_frames = np.nan, -1, 0

        row = dict(
            name=name, bond_idx=bond_idx, species=sp,
            xyz=str(xyz_path),
            conformer_policy=conformer_policy,
            max_frames=int(max_frames),
            E_Ha=float(E) if np.isfinite(E) else np.nan,
            best_frame=int(best_frame),
            n_frames=int(n_frames),
        )
        rows.append(row)
        done_keys.add(key)

        # incremental save after each species
        pd.DataFrame(rows).to_csv(per_species_csv, index=False)

print("Saved per-species table:", per_species_csv.resolve())
df_per = pd.DataFrame(rows)
df_per.tail()


In [ ]:
# ----------------------------
# Summarize to per-(name,bond) strain energies
# ----------------------------
HARTREE_TO_KCALMOL = 627.509474

df_per = pd.read_csv(per_species_csv)

# pivot energies
pivot = (df_per
         .pivot_table(index=["name","bond_idx"],
                      columns="species",
                      values="E_Ha",
                      aggfunc="min")
         .reset_index())

for sp in ["M","M_open","X","X_open"]:
    if sp not in pivot.columns:
        pivot[sp] = np.nan

pivot["E_strain_Ha"] = (pivot["M_open"] - pivot["M"]) - (pivot["X_open"] - pivot["X"])
pivot["E_strain_kcalmol"] = pivot["E_strain_Ha"] * HARTREE_TO_KCALMOL

# add settings (for provenance)
pivot["xc"] = xc
pivot["basis"] = basis
pivot["pcm_method"] = pcm_method
pivot["eps"] = eps
pivot["conformer_policy"] = conformer_policy
pivot["max_frames"] = int(max_frames)

pivot = pivot.sort_values(["name","bond_idx"]).reset_index(drop=True)
pivot.to_csv(summary_csv, index=False)
print("Saved summary:", summary_csv.resolve())

pivot


In [ ]:
# ----------------------------
# Quick plots
# ----------------------------
import matplotlib.pyplot as plt

df_sum = pd.read_csv(summary_csv)

# 1) Histogram of strain energies
plt.figure()
vals = df_sum["E_strain_kcalmol"].dropna().values
plt.hist(vals, bins=30)
plt.xlabel("E_strain (kcal/mol)")
plt.ylabel("count")
plt.title("CORE_SET DFT(+PCM) strain proxy distribution")
plt.show()

# 2) Per-molecule scatter by bond
for name, sub in df_sum.groupby("name"):
    plt.figure()
    plt.scatter(sub["bond_idx"], sub["E_strain_kcalmol"])
    plt.xlabel("bond_idx")
    plt.ylabel("E_strain (kcal/mol)")
    plt.title(f"{name}: strain proxy vs cut bond")
    plt.show()
